In [1]:
# Install Packages
!pip install transformers[sentencepiece] datasets evaluate rouge_score  py7zr -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.3/141.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.7/413.7 kB 12.0 MB/s eta 0:00:00
   ━━━━━━

In [2]:
# Which gpu we got
!nvidia-smi

Tue Feb 11 01:57:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
# import libraries
import evaluate
from transformers import pipeline
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import DataCollatorForSeq2Seq
from transformers import TrainingArguments, Trainer
from tqdm import tqdm
import torch

In [4]:
# check whether if i have gpy in my machine or not
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [5]:
# Define model checkpoint
model_ckpt = 'google/pegasus-cnn_dailymail'

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [6]:
# Initialize model
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [7]:
# Load Data
dataset_samsum = load_dataset('samsum')
dataset_samsum

README.md:   0%|          | 0.00/7.04k [00:00<?, ?B/s]

samsum.py:   0%|          | 0.00/3.36k [00:00<?, ?B/s]

The repository for samsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/samsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


corpus.7z:   0%|          | 0.00/2.94M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [8]:
# get rows in each dataset
split_lengths = [len(dataset_samsum[split]) for split in dataset_samsum]
split_lengths

[14732, 819, 818]

In [9]:
# get columns
print(f'features: {dataset_samsum["train"].column_names}')

features: ['id', 'dialogue', 'summary']


In [10]:
# Lets check dialogue & summary for any row
print('Dialogue: \n')

print(dataset_samsum['train'][1]['dialogue'])

print('Summary: \n')

print(dataset_samsum['train'][1]['summary'])

Dialogue: 

Olivia: Who are you voting for in this election? 
Oliver: Liberals as always.
Olivia: Me too!!
Oliver: Great
Summary: 

Olivia and Olivier are voting for liberals in this election. 


In [11]:
# Evaluation Pegasus on samsun data before fintune
dialogue = dataset_samsum['test'][0]['dialogue']
dialogue

"Hannah: Hey, do you have Betty's number?\nAmanda: Lemme check\nHannah: <file_gif>\nAmanda: Sorry, can't find it.\nAmanda: Ask Larry\nAmanda: He called her last time we were at the park together\nHannah: I don't know him well\nHannah: <file_gif>\nAmanda: Don't be shy, he's very nice\nHannah: If you say so..\nHannah: I'd rather you texted him\nAmanda: Just text him 🙂\nHannah: Urgh.. Alright\nHannah: Bye\nAmanda: Bye bye"

In [12]:
pipe = pipeline('summarization', model = model_ckpt)
pipe_out = pipe(dialogue)
pipe_out

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0
Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


[{'summary_text': "Amanda: Ask Larry Amanda: He called her last time we were at the park together .<n>Hannah: I'd rather you texted him .<n>Amanda: Just text him ."}]

In [13]:
# modify the generated summary
print(pipe_out[0]['summary_text'].replace(' .<n>','.\n'))

Amanda: Ask Larry Amanda: He called her last time we were at the park together.
Hannah: I'd rather you texted him.
Amanda: Just text him .


In [14]:
# split the dataset into smaller batches that we can process them simultaneously
def generate_batch_sized_chunks(list_of_elements, batch_size):
  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[i : i+batch_size]

def calculate_metric_on_test_dataset(dataset, metric, model, tokenizer,
                                batch_size=16, device = device,
                                column_text = 'dialogue',
                                column_summary = 'summary'):

  dialogue_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
  summary_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

  # looping through all batches
  for dialogue_batch, summary_batch in tqdm(zip(dialogue_batches, summary_batches), total = len(dialogue_batches)):

    # Apply tokenizer to like encode our text {ids}
    inputs = tokenizer(dialogue_batch, max_length = 1024, truncation = True, padding = 'max_length', return_tensors = 'pt')

    summaries = model.generate(input_ids = inputs['input_ids'].to(device),
                               attention_mask = inputs['attention_mask'].to(device),
                               length_penalty = 0.8, num_beams = 8, max_length = 128)

    decoded_summaries = [tokenizer.decode(summary, skip_special_tokens = True, clean_up_tokeniztion_spaces = True) for summary in summaries]

    decoded_summaries = [decode_summary.replace('',' ')for decode_summary in decoded_summaries]

    metric.add_batch(predictions = decoded_summaries, references= summary_batch)

  score = metric.compute()
  return score

In [15]:
# Check model performance
rouge_metric = evaluate.load("rouge")

score = calculate_metric_on_test_dataset(dataset_samsum['test'],
                                         rouge_metric,
                                         model_pegasus,
                                         tokenizer,
                                         column_text = 'dialogue',
                                         column_summary='summary',
                                         batch_size = 4)

100%|██████████| 205/205 [17:41<00:00,  5.18s/it]


In [16]:
rouge_names = ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']
rouge_dict = {rn: score[rn].fmeasure if hasattr(score[rn], 'fmeasure') else score[rn] for rn in rouge_names}

df = pd.DataFrame(rouge_dict, index=['pegasus_model'])
print(df)

                 rouge1    rouge2    rougeL  rougeLsum
pegasus_model  0.015563  0.000297  0.015525    0.01555


## Tokenize the Data

In [17]:
# Convert text to ids then to embedding
def covert_examples_to_features(example_batch):
  input_encodings = tokenizer(example_batch['dialogue'], max_length=1024, truncation = True)

  with tokenizer.as_target_tokenizer():
    target_encodings = tokenizer(example_batch['summary'], max_length=128, truncation = True)

  return {
      'input_ids': input_encodings['input_ids'],
      'attention_mask': input_encodings['attention_mask'],
      'labels': target_encodings['input_ids']
  }

dataset_samsum_encoded = dataset_samsum.map(covert_examples_to_features, batched = True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [18]:
print('hello')

hello


In [19]:
dataset_samsum_encoded['train'][0]

{'id': '13818513',
 'dialogue': "Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)",
 'summary': 'Amanda baked cookies and will bring Jerry some tomorrow.',
 'input_ids': [12195,
  151,
  125,
  7091,
  3659,
  107,
  842,
  119,
  245,
  181,
  152,
  10508,
  151,
  7435,
  147,
  12195,
  151,
  125,
  131,
  267,
  650,
  119,
  3469,
  29344,
  1],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [12195, 7091, 3659, 111, 138, 650, 10508, 181, 3469, 107, 1]}

In [20]:
# Handling Padding
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model = model_pegasus)

## FineTuning the Model

In [23]:
# set arguments

trainer_args = TrainingArguments(
    output_dir="pegasus-samsum",
    num_train_epochs=3,
    warmup_steps=300,  # Reduced from 500 to save memory
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=1e6,
    save_total_limit=2,  # Prevents excessive checkpoints
    gradient_accumulation_steps=8,  # Reduced from 16 to 8
    fp16=True,  # Enables mixed-precision training
    gradient_checkpointing=True,  # Saves memory by recomputing activations
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [24]:
trainer = Trainer(model = model_pegasus,
                  args = trainer_args,
                  tokenizer = tokenizer,
                  data_collator = seq2seq_data_collator,
                  train_dataset = dataset_samsum_encoded['train'],
                  eval_dataset = dataset_samsum_encoded['validation'])

<ipython-input-24-ecfebfaa68fd>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model = model_pegasus,


In [25]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: amr-abdelaty-545 (amr-abdelaty-545-october-6-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss,Validation Loss
500,1.698200,1.491231
1000,1.525500,1.427583
1500,1.485400,1.396009
2000,1.427800,1.379193
2500,1.448600,1.368241
3000,1.352500,1.357068
3500,1.369500,1.351907
4000,1.348300,1.351680
4500,1.327100,1.348041
5000,1.419600,1.344892


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=5523, training_loss=1.4691665198046207, metrics={'train_runtime': 12375.5459, 'train_samples_per_second': 3.571, 'train_steps_per_second': 0.446, 'total_flos': 1.6590071564869632e+16, 'train_loss': 1.4691665198046207, 'epoch': 2.9986424110779257})

In [ ]:
score = calculate_metric_on_test_dataset(
    dataset_samsum['test'], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'pegasus'] )

  0%|          | 0/410 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
 73%|███████▎  | 299/410 [31:24<11:23,  6.16s/it]

In [ ]:
## Save model
model_pegasus.save_pretrained("pegasus-samsum-model")

## Save tokenizer
tokenizer.save_pretrained("tokenizer")

## Evaluate the Model

In [ ]:
dataset_samsum = load_dataset("samsum")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("tokenizer")

In [ ]:
sample_text = dataset_samsum["test"][0]["dialogue"]

reference = dataset_samsum["test"][0]["summary"]

In [ ]:
gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}

pipe = pipeline("summarization", model="pegasus-samsum-model",tokenizer=tokenizer)

In [ ]:
print("Dialogue:")
print(sample_text)


print("\nReference Summary:")
print(reference)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])